# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [52]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [53]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cape town,-33.9258,18.4232,18.95,80,0,5.14,ZA,1673134240
1,1,imst,47.2684,10.6698,-0.85,78,63,2.73,AT,1673134324
2,2,airai,-8.9266,125.4092,19.68,73,95,1.23,TL,1673134325
3,3,kodiak,57.7900,-152.4072,-0.27,74,0,8.75,US,1673134326
4,4,mogadishu,2.0371,45.3438,25.26,85,15,7.72,SO,1673134112


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [54]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",    
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ["Humidity"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_good_weather = city_data_df.loc[(city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=27) & \
                                          (city_data_df["Wind Speed"]<=4.5),:]

# Drop any rows with null values
city_data_good_weather.count()
city_data_good_weather_no_null = city_data_good_weather.dropna(how='any')
city_data_good_weather_no_null.count()
# Display sample data
city_data_good_weather_no_null.head(17)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,puerto ayora,-0.7393,-90.3518,24.19,93,27,1.79,EC,1673134142
16,16,tuatapere,-46.1333,167.6833,21.21,41,3,2.68,NZ,1673134342
18,18,teacapan,22.5500,-105.7500,23.96,65,0,3.90,MX,1673134344
20,20,georgetown,5.4112,100.3354,24.97,88,20,2.06,MY,1673134347
27,27,libreville,0.3925,9.4537,25.53,81,80,2.20,GA,1673134355
29,29,new norfolk,-42.7826,147.0587,22.73,41,0,1.34,AU,1673134358
30,30,lamu,-2.2717,40.9020,26.12,85,18,4.25,KE,1673134359
36,36,nabire,-3.3667,135.4833,25.82,82,100,0.94,ID,1673134172
39,39,matara,5.9485,80.5353,22.63,91,82,1.80,LK,1673134374
40,40,bengkulu,-3.8004,102.2655,23.82,87,96,2.55,ID,1673134376


### Step 3: Create a new DataFrame called `hotel_df`.

In [58]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df_all_column=city_data_good_weather_no_null.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df_all_column["Hotel Name"]=""
hotel_df=hotel_df_all_column[["City","Country","Lat","Lng","Humidity","Hotel Name"]]
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,puerto ayora,EC,-0.7393,-90.3518,93,
16,tuatapere,NZ,-46.1333,167.6833,41,
18,teacapan,MX,22.5500,-105.7500,65,
20,georgetown,MY,5.4112,100.3354,88,
27,libreville,GA,0.3925,9.4537,81,
...,...,...,...,...,...,...
540,mersing,MY,2.4312,103.8405,94,
542,vallenar,CL,-28.5708,-70.7581,35,
551,acayucan,MX,17.9500,-94.9167,83,
557,say,NE,13.1007,2.3689,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "categories":"accommodation.hotel",
          "apiKey":geoapify_key          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


D:\My App\Anaconda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
tuatapere - nearest hotel: No hotel found
teacapan - nearest hotel: illas y Hotel Maria Fernanda
georgetown - nearest hotel: Page 63 hostel
libreville - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
nabire - nearest hotel: Hotel Anggrek
matara - nearest hotel: Villa Palm Breeze
bengkulu - nearest hotel: Dena Hotel
pedernales - nearest hotel: Hostal Doña Chava
pochutla - nearest hotel: Posada San Jose
wagga wagga - nearest hotel: The Astor
mataura - nearest hotel: No hotel found
nampula - nearest hotel: Hotel Lurio
zabid - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
nioro - nearest hotel: No hotel found
augusto correa - nearest hotel: No hotel found
wamba - nearest hotel: No hotel found
bambous virieux - nearest hotel: Otentic Eco Tent Experience
cape coast - nearest hotel: Mighty Victory Hotel
ma

,City,Country,Lat,Lng,Humidity,Hotel Name
10,puerto ayora,EC,-0.7393,-90.3518,93,Hostal La Mirada De Solitario George
16,tuatapere,NZ,-46.1333,167.6833,41,No hotel found
18,teacapan,MX,22.5500,-105.7500,65,illas y Hotel Maria Fernanda
20,georgetown,MY,5.4112,100.3354,88,Page 63 hostel
27,libreville,GA,0.3925,9.4537,81,No hotel found
...,...,...,...,...,...,...
540,mersing,MY,2.4312,103.8405,94,Mersing Inn Hotel
542,vallenar,CL,-28.5708,-70.7581,35,Hotel del Marqués
551,acayucan,MX,17.9500,-94.9167,83,Arcos del Parque
557,say,NE,13.1007,2.3689,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 1000,
    scale = 0.1,
    color = "City",
    hover_cols= ["Humidity","Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)